# Decision Generator (Anchored, Recency-Aware, TAST-Driven, Risk-Capped)

This notebook generates the next-round decision files:
- `data/prices/prices_<vNext>.json`
- `data/amounts/amounts_<vNext>.json`
- `data/schedules/schedules_<vNext>.json`

It reads `model_data.json` (from the Model & Analysis Generator) and applies:
- **Anchor pricing from the best-selling week** (per product)
- **TAST_norm** (time-adjusted sell-through) for demand intensity
- **Recency decay** (penalize items with long time since last sale)
- **Dead-item suppression** (stronger reductions for zero recent sales)
- **Risk-capped amounts** (avoid expensive overstock)
- **Schedule by ROI** (revenue / staff cost, once per version)

Exports an **audit CSV** and prints quick anomaly checks.

In [ ]:
# === Setup & Parameters ======================================================
import json, math
from pathlib import Path
import numpy as np
import pandas as pd

# Paths
DATA_DIR = Path("data")
SCHEDULES_DIR = DATA_DIR / "schedules"
OUT_DIR = Path("output"); OUT_DIR.mkdir(exist_ok=True, parents=True)

MODEL_DATA_PATH = Path("model_data.json")  # produced by model_and_analysis_generator

# Core parameters (conservative for competitive play)
PRICE_UP_HOT      = 0.03   # +3% if very hot (TAST high)
PRICE_UP_GOOD     = 0.02   # +2% if strong demand
PRICE_DOWN_WEAK   = 0.08   # -8% if weak demand with stock
PRICE_DOWN_DEAD   = 0.15   # -15% if dead item

MIN_MARGIN_RATE   = 0.10   # ensure price >= (1 + MIN_MARGIN_RATE) * supplier_price
ROUND_CAP_STD     = 0.05   # ±5% per round cap
ROUND_CAP_DEAD    = 0.10   # ±10% for dead items

AMT_BASE_BUFFER   = 1.20   # base buffer over max observed weekly demand
AMT_BOOST_HOT     = 1.20   # extra +20% for very hot items
AMT_DEAD_FACTOR   = 0.50   # dead items keep 50% of average stock
RISK_CAP_MIN      = 1.10   # min cap multiplier on avg amount
RISK_CAP_MAX      = 1.60   # max cap multiplier on avg amount

EMA_ALPHA         = 0.6    # recency emphasis
RECENCY_DECAY     = 0.10   # 10% penalty per week since last sale (clamped)
RECENCY_MIN       = 0.60   # floor 60%

# TAST thresholds (tune if needed)
TAST_HOT          = 0.90   # sold out early / very strong
TAST_GOOD_LOW     = 0.65   # good demand lower bound
TAST_WEAK_HIGH    = 0.25   # weak demand upper bound

# Optional: product name filters for quick audits
ANOMALY_FILTERS   = ["dinosaur", "mattress"]

In [ ]:
# === Load model data + helpers ==============================================
assert MODEL_DATA_PATH.exists(), "model_data.json not found. Run model_and_analysis_generator first."
md = pd.DataFrame(json.load(open(MODEL_DATA_PATH, "r", encoding="utf-8")))

VERSIONS = sorted(md["version"].unique())
V_NOW  = VERSIONS[-1]
HIST_VERS = [v for v in VERSIONS if v < V_NOW]
V_NEXT = V_NOW + 1

print("Versions:", VERSIONS, "| Learn from:", HIST_VERS, "| Next:", V_NEXT)

def safe_div(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)

def ema_series(x, alpha=EMA_ALPHA):
    s = None
    for xi in x:
        s = alpha*xi + (1-alpha)*(s if s is not None else xi)
    return s if s is not None else 0.0

In [ ]:
# === Build historical table & select anchors =================================
# Aggregate core facts per (version, product)
g = (md.groupby(["version","product"], as_index=False)
        .agg(qty=("qty","sum"),
             amount=("amount","mean"),
             price=("price","mean"),
             revenue=("revenue","sum"),
             supplier_price=("supplier_price","mean"),
             TAST_norm=("TAST_norm","mean")))  # TAST_norm now available

g_hist = g[g["version"].isin(HIST_VERS)].copy()

# Anchor: best-selling week by qty, then higher revenue, then lower price
g_sorted = g_hist.sort_values(["product","qty","revenue","price"],
                              ascending=[True, False, False, True])
anchor = (g_sorted.groupby("product", as_index=False).first()
                 [["product","version","price","qty","revenue","amount","supplier_price"]]
                 .rename(columns={"version":"anchor_v","price":"p_anchor",
                                  "qty":"anchor_qty","revenue":"anchor_revenue",
                                  "amount":"anchor_amount",
                                  "supplier_price":"supplier_price_ref"}))

print("Anchor sample:")
display(anchor.head(8))

In [ ]:
# === Indicators: TAST-EMA, VOL, Risk, Recency ================================
# Per-product historical indicators
hist = (g_hist.groupby("product")
          .apply(lambda d: pd.Series({
              # Use TAST_norm (time-aware sell-through) as the demand signal
              "TAST_EMA": ema_series(d["TAST_norm"].fillna(0.0), alpha=EMA_ALPHA),
              "qty_max": d["qty"].max(),
              "amount_avg": d["amount"].mean(),
              "VOL": float(d["qty"].std(ddof=0)) if len(d) > 1 else 0.0
          }))
          .reset_index())

# Medians for risk scaling
p_med = g_hist.groupby("product")["price"].median().rename("price_med")
c_med = g_hist.groupby("product")["supplier_price"].median().rename("cost_med")

ref = (anchor.merge(hist, on="product", how="left")
             .merge(p_med, on="product", how="left")
             .merge(c_med, on="product", how="left"))

ref["PCR"]     = safe_div(ref["p_anchor"], ref["supplier_price_ref"])
ref["PCR_med"] = safe_div(ref["price_med"], ref["cost_med"])

# Risk score: higher → more cautious amounts
w1, w2, w3 = 0.5, 0.3, 0.2
VOL_med = ref["VOL"].median() if ref["VOL"].median() > 0 else 1.0
ref["RISK"] = (
    (ref["PCR"] / ref["PCR_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w1 +
    (ref["p_anchor"] / ref["price_med"]).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w2 +
    (ref["VOL"] / VOL_med).replace([np.inf, -np.inf], np.nan).fillna(1.0)*w3
)

# Dead items: no sales in last 2 historical weeks (if exist)
last_weeks = sorted(HIST_VERS)[-2:] if len(HIST_VERS) >= 2 else HIST_VERS
recent = g_hist[g_hist["version"].isin(last_weeks)].groupby("product")["qty"].sum()
ref = ref.merge(recent.rename("qty_recent"), on="product", how="left").fillna({"qty_recent":0.0})
ref["is_dead"] = ref["qty_recent"] <= 0

# Recency: weeks since last non-zero sale across all versions
last_sale_v = (g[g["qty"]>0].groupby("product")["version"].max().rename("last_sold_version"))
ref = ref.merge(last_sale_v, on="product", how="left")
ref["weeks_since_sale"]   = V_NOW - ref["last_sold_version"].fillna(0)
ref["recency_multiplier"] = (1.0 - RECENCY_DECAY*ref["weeks_since_sale"]).clip(lower=RECENCY_MIN, upper=1.0)

print("Indicators sample:")
display(ref.head(8))

In [ ]:
# === Price rule (anchored + TAST/recency/dead guards) ========================
def price_next(r):
    p0   = float(r["p_anchor"] or 0.0)
    sp   = float(r["supplier_price_ref"] or 0.0)
    tast = float(r["TAST_EMA"] or 0.0)   # time-aware demand
    dead = bool(r["is_dead"])
    rec  = float(r["recency_multiplier"] or 1.0)

    # anchored default
    p = p0
    if dead:
        p = p0 * (1.0 - PRICE_DOWN_DEAD)
    elif tast >= TAST_HOT:
        p = p0 * (1.0 + PRICE_UP_HOT)
    elif TAST_GOOD_LOW <= tast < TAST_HOT:
        p = p0 * (1.0 + PRICE_UP_GOOD)
    elif tast <= TAST_WEAK_HIGH and (r["amount_avg"] or 0) > 0:
        p = p0 * (1.0 - PRICE_DOWN_WEAK)

    # Recency decay (penalize stale items)
    p *= rec

    # Margin floor
    min_allowed = sp*(1.0 + MIN_MARGIN_RATE) if sp>0 else 0.0
    p = max(p, min_allowed)

    # Per-round caps (tighter for living items, looser for dead)
    cap_up   = p0*(1.0 + (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    cap_down = p0*(1.0 - (ROUND_CAP_DEAD if dead else ROUND_CAP_STD))
    p = min(max(p, cap_down), cap_up)

    return round(float(p), 2)

In [ ]:
# === Amount rule (TAST + risk cap + recency/dead) ============================
def amount_next(r):
    tast    = float(r["TAST_EMA"] or 0.0)
    qty_max = float(r["qty_max"] or 0.0)
    amt_avg = float(r["amount_avg"] or 0.0)
    risk    = float(r["RISK"] or 1.0)
    dead    = bool(r["is_dead"])
    rec     = float(r["recency_multiplier"] or 1.0)

    # Base: over observed max demand
    base = math.ceil(qty_max * AMT_BASE_BUFFER)
    if tast > 0.95:
        base = math.ceil(base * AMT_BOOST_HOT)
    if dead:
        base = max(0, round(amt_avg * AMT_DEAD_FACTOR))

    # Risk cap: higher risk => smaller cap multiplier
    cap_mult = np.clip(1.0 + 0.5/(risk if risk>0 else 1.0), RISK_CAP_MIN, RISK_CAP_MAX)
    cap = amt_avg * cap_mult
    if cap > 0:
        base = min(base, math.ceil(cap))

    # Recency downscale
    base = int(base * rec)

    return int(max(0, base))

In [ ]:
# === Schedule selection by ROI ==============================================
# Use md: version-level revenue and staff cost (avoid double-count: take one staff cost per version)
rev_by_v   = md.groupby("version")["revenue"].sum()
staff_by_v = md.groupby("version")["version_staff_cost"].max()  # single value per version
roi = (rev_by_v / staff_by_v.replace(0, np.nan)).dropna()

# Consider only historical weeks
roi_hist = roi[roi.index.isin(HIST_VERS)]
if not roi_hist.empty:
    sched_src_v = int(roi_hist.idxmax())
else:
    # fallback to latest historical with a schedule file
    candidates = [v for v in reversed(HIST_VERS) if (SCHEDULES_DIR / f"schedules_{v}.json").exists()]
    sched_src_v = candidates[0] if candidates else HIST_VERS[-1]

with open(SCHEDULES_DIR / f"schedules_{sched_src_v}.json", "r", encoding="utf-8") as f:
    schedules_next = json.load(f)

print(f"Schedule source version (best ROI among history): v{sched_src_v}")

In [ ]:
# === Generate JSONs & Audit ==================================================
prices_next  = {r["product"]: price_next(r)  for _, r in ref.iterrows()}
amounts_next = {r["product"]: amount_next(r) for _, r in ref.iterrows()}

# Write decisions
(DATA_DIR/"prices").mkdir(parents=True, exist_ok=True)
(DATA_DIR/"amounts").mkdir(parents=True, exist_ok=True)
(DATA_DIR/"schedules").mkdir(parents=True, exist_ok=True)

with open(DATA_DIR/"prices"/f"prices_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(prices_next, f, indent=2, ensure_ascii=False)
with open(DATA_DIR/"amounts"/f"amounts_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(amounts_next, f, indent=2, ensure_ascii=False)
with open(DATA_DIR/"schedules"/f"schedules_{V_NEXT}.json", "w", encoding="utf-8") as f:
    json.dump(schedules_next, f, indent=2, ensure_ascii=False)

# Audit table
audit = ref[["product","anchor_v","p_anchor","qty_max","amount_avg","TAST_EMA","RISK","is_dead","weeks_since_sale","recency_multiplier"]].copy()
audit["price_next"]  = audit["product"].map(prices_next)
audit["amount_next"] = audit["product"].map(amounts_next)
audit["d_price_vs_anchor_%"] = (audit["price_next"] - audit["p_anchor"]) / audit["p_anchor"] * 100

audit_path = OUT_DIR / f"decision_v{V_NEXT}_audit.csv"
audit.to_csv(audit_path, index=False)

print("Generated decisions:")
print(" -", DATA_DIR/"prices"/f"prices_{V_NEXT}.json")
print(" -", DATA_DIR/"amounts"/f"amounts_{V_NEXT}.json")
print(" -", DATA_DIR/"schedules"/f"schedules_{V_NEXT}.json")
print("Audit:", audit_path)

display(audit.sort_values(["is_dead","weeks_since_sale","RISK"], ascending=[False, False, False]).head(15))

In [ ]:
# === Quick anomaly checks ====================================================
def show_anomalies(filters=ANOMALY_FILTERS, df=audit):
    if not filters:
        print("No filters provided."); return
    flt = None
    for token in filters:
        m = df["product"].str.contains(token, case=False, na=False)
        flt = m if flt is None else (flt | m)
    res = df[flt] if flt is not None else pd.DataFrame()
    if res.empty:
        print("No products matched filters:", filters)
    else:
        print("Anomaly candidates:")
        display(res.sort_values(["is_dead","weeks_since_sale","RISK"], ascending=[False, False, False]))

show_anomalies()

### Notes & Rationale

- **TAST_norm** (time-adjusted sell-through) replaces raw sell-through as primary demand signal.
- **Anchor pricing** avoids using stale or depressed last-week prices; we anchor to the best-selling week.
- **Recency decay** reduces both price and amount for items with a long time since last sale.
- **Dead-item suppression** applies stronger reductions and lower restock targets.
- **Risk cap** prevents big stock increases on expensive or volatile items.
- **Schedule ROI** picks the historical week with the best revenue/staff-cost ratio.

Per-round caps keep changes conservative in a competitive environment.
If any product looks odd in the audit, tune:
- `RECENCY_DECAY` (0.05–0.15), `RECENCY_MIN` (0.5–0.8),
- amount caps (`RISK_CAP_MIN/MAX`),
- price caps (`ROUND_CAP_STD/DEAD`),
- TAST thresholds (`TAST_HOT`, `TAST_GOOD_LOW`, `TAST_WEAK_HIGH`).